## Install cobrapy

In [1]:
%pip install cobra --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 24.6 MB/s eta 0:00:00


## Import CAM model from Github

In [2]:
# Retrieving the model
!wget -q https://raw.githubusercontent.com/sshameer/DielLeafModeling/refs/heads/main/C3_model.sbml

## Import functions required to read sbml, write sbml and perform pFBA

In [3]:
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model
from cobra.flux_analysis import pfba

## Import the model and constrain diel leaf model for CAM photosynthesis

In [6]:
C3_model = read_sbml_model("C3_model.sbml")
CAM_model = C3_model.copy()

In [7]:
# restricting CO2 uptake to night
CAM_model.reactions.CO2_tx1.upper_bound = 0
CAM_model.reactions.CO2_tx1.lower_bound = 0



# setting photon uptake
PPFD = 800
CAM_model.reactions.Photon_tx1.upper_bound = PPFD
CAM_model.reactions.Photon_tx1.lower_bound = 0

# setting the maintenance cost
VATPase = 0.0049*PPFD+2.7851
CAM_model.reactions.get_by_id("ATPase_tx1").bounds= (VATPase,VATPase)
solution = pfba(CAM_model)

In [8]:
CAM_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e2,CO2_tx2,42.52,1,100.00%
WATER_e1,H2O_tx1,19.11,0,0.00%
WATER_e2,H2O_tx2,20.01,0,0.00%
NITRATE_e1,Nitrate_tx1,0.2773,0,0.00%
NITRATE_e2,Nitrate_tx2,0.1848,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,6.115,0,0.00%
Photon_e1,Photon_tx1,800,0,0.00%
SULFATE_e1,SO4_tx1,0.003533,0,0.00%
PROTON_c1,unlProtHYPO_c1,0.4024,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux


### Note that the CO2 is taken up via CO2_tx2, where the 2 tag stands for night-time.